In [42]:
from functools import partial
import numpy as np
import os
import pandas as pd
import scipy
import glob
import pickle
import bz2

import seaborn as sns
sns.set_style("whitegrid")
import matplotlib as mpl
import matplotlib.pyplot as plt


import sys
sys.path.append('../')
import util

save_fig = partial(util.save_fig, save=True)

In [43]:
results_dir = os.path.join(util.ANALYSIS_DIR, 'D3_gmm')
diffGMM_files = glob.glob(os.path.join(results_dir, 'diffGMM*.pickle.bz2'))
diffGMM_files

['/home/drake/workspace/em-2020/analysis/D3_gmm/diffGMM_Rafiq2016-CTS_subset.pickle.bz2',
 '/home/drake/workspace/em-2020/analysis/D3_gmm/diffGMM_Rafiq2016-CTS-TI-KAOPEN_subset.pickle.bz2',
 '/home/drake/workspace/em-2020/analysis/D3_gmm/diffGMM_Rafiq2016-CTS-TI_subset.pickle.bz2',
 '/home/drake/workspace/em-2020/analysis/D3_gmm/diffGMM_Rafiq2016_subset.pickle.bz2']

In [49]:
def get(f):
    with bz2.BZ2File(f) as fd:
        return pickle.load(fd)


default = get('/home/drake/workspace/em-2020/analysis/D3_gmm/template_output.pickle.bz2')
rafiq = get('/home/drake/workspace/em-2020/analysis/D3_gmm/diffGMM_Rafiq2016_subset.pickle.bz2')
cts = get('/home/drake/workspace/em-2020/analysis/D3_gmm/diffGMM_Rafiq2016-CTS_subset.pickle.bz2')
cts_ti = get('/home/drake/workspace/em-2020/analysis/D3_gmm/diffGMM_Rafiq2016-CTS-TI_subset.pickle.bz2')
cts_ti_kaopen = get('/home/drake/workspace/em-2020/analysis/D3_gmm/diffGMM_Rafiq2016-CTS-TI-KAOPEN_subset.pickle.bz2')

pd.testing.assert_frame_equal(default['pdynmc_coef'], cts_ti_kaopen['pdynmc_coef'])
check = pd.concat([rafiq['pdynmc_coef'], cts_ti_kaopen['pdynmc_coef']], axis='columns')
check.loc[check.index.str.startswith('L')]

,Estimate,Std.Err.rob,z.value.rob,Pr...z.rob..,Estimate,Std.Err.rob,z.value.rob,Pr...z.rob..
L1.ENI,0.092050,0.877438,0.105,0.91638,0.198553,0.082621,2.403,0.01626
L2.ENI,0.149892,0.297147,0.504,0.61426,0.228045,0.095956,2.377,0.01745
L0.POP,-0.064137,0.365662,-0.175,0.86108,-0.087448,0.039891,-2.192,0.02838
L0.AFL,0.006458,0.049196,0.131,0.89578,0.015900,0.007848,2.026,0.04276
L0.AFL2,-0.029028,0.007566,-3.837,0.00012,-0.011592,0.015626,-0.742,0.45809
L0.URB,-0.028907,0.168108,-0.172,0.86344,-0.048191,0.023224,-2.075,0.03799
L0.TS,-0.054675,0.081610,-0.670,0.50286,NaN,NaN,NaN,NaN
L0.CTS,NaN,NaN,NaN,NaN,-0.303107,0.261844,-1.158,0.24686
L0.TI,NaN,NaN,NaN,NaN,-0.108752,0.119294,-0.912,0.36177
L0.KAOPEN,NaN,NaN,NaN,NaN,0.017005,0.015195,1.119,0.26314


In [57]:
coefs = rafiq['pdynmc_coef']
col_map = {'Estimate': 'estimate', 'Std.Err.rob': 'std. error', 'Pr...z.rob..': 'p-value'}

var_map = {x:x.replace('L0.', '') for x in coefs.index if x.startswith('L0')}
coefs.loc[var_map.keys(), col_map.keys()]
coefs.rename(columns=col_map)


Index(['Estimate', 'Std.Err.rob', 'z.value.rob', 'Pr...z.rob..'], dtype='object')

In [ ]:
def resid_plot(data):
    sns.regplot(x='fitted values', y='residuals', 
                data=data.rename(columns={'fitted': 'fitted values'}))
    plt.axhline(0, color='gray')

In [ ]:
resid_plot(rafiq['pdynmc_resid'])
resid_plot(cts['pdynmc_resid'])
resid_plot(cts_ti['pdynmc_resid'].iloc[0:50])

In [ ]:
pd.concat([cts_ti_kaopen['pdynmc_coef'], default['pdynmc_coef']], axis='columns')